## Language model for error row finder

In [7]:
from __future__ import generator_stop
import numpy as np
import random
import nltk, re, string, collections
from nltk.util import ngrams
import pandas as pd
from nltk.probability import *
import string
import copy
from sklearn.utils import shuffle

# li = ['timestamp', 'description','limit6','mailid','float']
# temp = pd.read_csv("./dataset.csv")
temp = pd.read_csv("./reld.csv", delimiter='|', header=None, low_memory=False)
temp.columns = temp.columns.astype(str)
temp = temp.fillna("")
temp.to_csv("./input.csv")
tempn = pd.DataFrame()
for i in temp.columns:
    tempn[str(i)] = '*' + temp[str(i)].map(str) + '*'
le80 = int(0.8 * len(tempn))
npdf = copy.deepcopy(tempn[:le80])
tedf = copy.deepcopy(tempn[le80:])



def map_parameter_init():
    d = {}
    pun = list(string.punctuation)
    num = list(string.digits)
    let = list(string.ascii_letters)
    for i in pun:
        d[i] = i
    for j in num:
        d[j] = 'n'
    for k in let:
        d[k] = 'c'
    d[' '] = ' '
    d['\n'] = '\n'
    d['\t'] = '\t'
    return d

d = map_parameter_init()
vi = dict()
cfd = dict()
mingram = dict()
total = dict()
cols = temp.columns
lin = {key:[] for key in cols}
rejli = {key:[] for key in cols}
rejrli ={key:[] for key in cols}
rejpd = pd.DataFrame(columns=temp.columns, index = range(300)) #.reindex_like(temp)

In [2]:
def mapval(a, i):
    global lin
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    lin[str(i)].append(te)

def trainmode():
    for i in cols:
        #distribution counting
        tokd = pd.DataFrame(npdf[str(i)].map(str).apply(list))
        val = tokd[str(i)].apply(mapval,args = (i,))
        vi[str(i)] = collections.Counter(lin[str(i)])

        ##Language model implementation
        toke = list(npdf[str(i)].str.cat(sep='\n'))
        minv = npdf[str(i)].map(len).min()
        mingram[str(i)] = minv
        u,inv = np.unique(toke,return_inverse = True)
        te = np.array([d[x] for x in u])[inv].reshape(len(toke))
        total[str(i)] = len(te)
        bigr = nltk.ngrams(te,mingram[str(i)])
        condition_pairs = (((w[:-1]), w[-1]) for w in bigr)
        cfd[str(i)] = nltk.ConditionalFreqDist(condition_pairs)
        print(total[str(i)])
    print(cfd)


trainmode()

2132185
3063049
9645875
8683599
2906211
1837829
1745034
3554568
1225219
2940393
7351319
1621316
1531524
1531524
2326402
2261705
{'0': <ConditionalFreqDist with 14 conditions>, '1': <ConditionalFreqDist with 10 conditions>, '2': <ConditionalFreqDist with 151 conditions>, '3': <ConditionalFreqDist with 164 conditions>, '4': <ConditionalFreqDist with 8 conditions>, '5': <ConditionalFreqDist with 6 conditions>, '6': <ConditionalFreqDist with 8 conditions>, '7': <ConditionalFreqDist with 84 conditions>, '8': <ConditionalFreqDist with 4 conditions>, '9': <ConditionalFreqDist with 26 conditions>, '10': <ConditionalFreqDist with 24 conditions>, '11': <ConditionalFreqDist with 30 conditions>, '12': <ConditionalFreqDist with 8 conditions>, '13': <ConditionalFreqDist with 9 conditions>, '14': <ConditionalFreqDist with 8 conditions>, '15': <ConditionalFreqDist with 115 conditions>}


In [ ]:
cfd['0'].items()

In [3]:
#run on actual data
def tmapval(a, va,i):
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    try:
        tel = vi[str(i)].get(te)/va
    except:
        rejli[str(i)].append(te)
        rejrli[str(i)].append(a)
        
def feature_mapping():
    count = 0
    for i in cols:
        ind = 0
        tdf = pd.DataFrame(tedf[str(i)].map(str).apply(list))
        count = len(tdf.index)
        tempa = tdf[str(i)].apply(tmapval, args = (count,i))
        print("Number of rejected items in",str(i),"is", len(rejli[str(i)]),"\n")
        if rejli[str(i)] !=[]:
            for j in rejli[str(i)]:
                tr = nltk.ngrams(j,mingram[str(i)])
                if list(tr) == []:
                    vi = 0
                    tem = copy.deepcopy(rejrli[str(i)][rejli[str(i)].index(j)])
                    tem.pop(0)
                    tem.pop(-1)
                    rejpd.at[ind, str(i)] = str("".join(tem))
                    ind+=1
                    continue
                else:
                    tr = nltk.ngrams(j,mingram[str(i)])
                    cpairs = (((w[:-1]), w[-1]) for w in tr)
                    co = 0
                    vi = 0
                    for k in list(cpairs):
                        v = cfd[str(i)][k[0]][k[1]]
                        if co == 0:
                            vi = v/total[str(i)]
                            co = 1
                        else:
                            vi *= v/total[str(i)]
                    if vi ==0:
                        tel = copy.deepcopy(rejrli[str(i)][rejli[str(i)].index(j)])
                        tel.pop(0)
                        tel.pop(-1)
                        rejpd.at[ind, str(i)] = str("".join(tel))
                        ind+=1
                    elif vi!=0:
                        pass
#                         print("Accepted after language mode:","".join(rejrli[str(i)][rejli[str(i)].index(j)]))
                
x = feature_mapping()

Number of rejected items in 0 is 8 

Number of rejected items in 1 is 0 

Number of rejected items in 2 is 17746 

Number of rejected items in 3 is 285 

Number of rejected items in 4 is 0 

Number of rejected items in 5 is 0 

Number of rejected items in 6 is 0 

Number of rejected items in 7 is 474 

Number of rejected items in 8 is 0 

Number of rejected items in 9 is 0 

Number of rejected items in 10 is 0 

Number of rejected items in 11 is 59 

Number of rejected items in 12 is 0 

Number of rejected items in 13 is 0 

Number of rejected items in 14 is 0 

Number of rejected items in 15 is 101 



In [6]:
rejpd.to_csv("./output.csv")

In [ ]:
# condition_pairs.throw()
# a = {}
# k = 0
# while k < 10:
#     <dynamically create key> 
#     key = ...
#     <calculate value> 
#     value = ...
#     a[key] = value 
#     k += 1

# t = nltk.ngrams("timestamp,description,ipaddress,limit6,float,year,money,country,Enum,mailid,bool,longnumber,stockprice,output", 3)
# # list(t)
# for i in list(t):
#     print(i)
#     print(i[:-1])
#     print(i[-1])
    
# t = nltk.ngrams("timestamp,description,ipalongnumber,stockprice,output", 3)
# if list(t) == []:
#     print("in")
# else:
#     print("out")

    
# # collections.Counter(vi.most_common(1))
# l = list(vi.keys())
# for i in l:
#     print(collections.Counter(i))
# npdf['mailid'].map(len).sum()/len(npdf['mailid'].index) $%average length of list in a columnx
# fdist = nltk.FreqDist(w.lower() for w in te)
# total = 0
# for word in fdist:
#     total += fdist[word]
    
# v = list('**nncn')
# for w in v:
#     if w in fdist:
#         print(w,'freq',fdist[w],'prob',fdist[w]/total)
#     else:
#         print(w,'freq',0,'prob',0)
# cfd.items()
# cfd.keys()
# cfd[v[0]][v[1]]

# toke = list(data)
# u,inv = np.unique(toke,return_inverse = True)
# te = np.array([d[x] for x in u])[inv].reshape(len(toke))
# esBigrams = ngrams(te, 3)
# # list(esBigrams)[:10]
# esBigramFreq = collections.Counter(esBigrams)
# esBigramFreq.most_common(20)
